## Instagram Scrapper

In [2]:
!pip install textblob

### Downloading data from Instagram users

In [5]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import requests
import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup     #html parser
import ssl
import json


class Insta_Info_Scraper:

    def getinfo(self, url):
        html = urllib.request.urlopen(url, context=self.ctx).read()
        soup = BeautifulSoup(html, 'html.parser')
        #data = get the information from the tag og:description containing information about followers, posts etc. 
        data = soup.find_all('meta', attrs={'property': 'og:description'    
                             })
        text = data[0].get('content').split()
        user = '%s %s %s' % (text[-3], text[-2], text[-1])
        followers = text[0]
        following = text[2]
        posts = text[4]
        print ('User:', user)
        print ('Followers:', followers)
        print ('Following:', following)
        print ('Posts:', posts)
        print ('---------------------------')

    def main(self):
        self.ctx = ssl.create_default_context()
        self.ctx.check_hostname = False
        self.ctx.verify_mode = ssl.CERT_NONE

        with open('users.txt') as f:
            self.content = f.readlines()
        self.content = [x.strip() for x in self.content]
        for url in self.content:
            self.getinfo(url)


if __name__ == '__main__':
    obj = Insta_Info_Scraper()
    obj.main()

User: videos they post.
Followers: Follow
Following: (@lgbtq)
Posts: never
---------------------------
User: from LGBT (@lgbt)
Followers: 529k
Following: 88
Posts: 378
---------------------------
User: from Pride (@pride)
Followers: 343.1k
Following: 85
Posts: 320
---------------------------


### Downloading publications (text and pictures) containing some hashtags
#### We also calculate the average sentiment of the downloaded publications

In [3]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import requests
import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl
import json
from textblob import TextBlob


class Insta_Image_Links_Scraper:

    def getlinks(self, hashtag, url):

        cummulative_polarity = 0
        cummulative_subjectivity = 0
        counter = 0
        
        html = urllib.request.urlopen(url, context=self.ctx).read()
        soup = BeautifulSoup(html, 'html.parser')
        script = soup.find('script', text=lambda t: \
                           t.startswith('window._sharedData'))
        page_json = script.text.split(' = ', 1)[1].rstrip(';')
        data = json.loads(page_json)
        print ('Scraping links with #' + hashtag+"...........")
        for post in data['entry_data']['TagPage'][0]['graphql'
                ]['hashtag']['edge_hashtag_to_media']['edges']:
            image_src = post['node']['thumbnail_resources'][1]['src']
            if (post['node']['edge_media_to_caption']['edges'] != []):
                text = post['node']['edge_media_to_caption']['edges'][0]['node']['text']
                #print(text) # text of the post
                
                # getting the polarity of the sentimentalisis
                analysis = TextBlob(text)
                polarity = analysis.sentiment[0]
                cummulative_polarity += polarity
                subjectivity = analysis.sentiment[1]
                cummulative_subjectivity += subjectivity
                counter += 1
            hs = open(hashtag + '.txt', 'a')
            hs.write(image_src + '\n')
            hs.close()
        if counter != 0:
            print ('Avg Subjectivity', cummulative_subjectivity/counter)
            print ('Avg Polarity', cummulative_polarity/counter)
            print ('Counter', counter)

    def main(self):
        self.ctx = ssl.create_default_context()
        self.ctx.check_hostname = False
        self.ctx.verify_mode = ssl.CERT_NONE

        with open('hashtag_list.txt') as f:
            self.content = f.readlines()
        self.content = [x.strip() for x in self.content]
        for hashtag in self.content:
            self.getlinks(hashtag,
                          'https://www.instagram.com/explore/tags/'
                          + hashtag + '/')


if __name__ == '__main__':
    obj = Insta_Image_Links_Scraper()
    obj.main()

Scraping links with #LGBTQ...........
Avg Subjectivity 0.4280669588592693
Avg Polarity 0.2347597646648244
Counter 66
Scraping links with #LGBT...........
Avg Subjectivity 0.517523679865683
Avg Polarity 0.2757294348594814
Counter 70
Scraping links with #lgbtq...........
Avg Subjectivity 0.4280669588592693
Avg Polarity 0.2347597646648244
Counter 66
Scraping links with #lgbt...........
Avg Subjectivity 0.517523679865683
Avg Polarity 0.2757294348594814
Counter 70
